<a href="https://colab.research.google.com/github/slv-ai/Fine-Tune-LLMs-with-DPO/blob/main/DPO_phi2_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q torch datasets accelerate transformers datasets peft bitsandbytes trl einops tqdm #wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
!pip install huggingface_hub -q -U
from huggingface_hub import notebook_login
notebook_login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 7.2 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,TrainingArguments,BitsAndBytesConfig
from trl import DPOTrainer
from peft import LoraConfig, PeftModel,prepare_model_for_kbit_training,get_peft_model
from datasets import load_dataset
from huggingface_hub import notebook_login

Dataset

In [ ]:
dpo_dataset=load_dataset("json",data_files="dpo_dataset.json",split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def chat_format(example):
  if len(example["prompt"])>0:
    prompt="<|im_start|>system\nYou are an AI therapist. You will be given a task. You must generate an response for the user query.<|im_end|>\n<|im_start|>User\n"+example["prompt"]+"<|im_end|>\n<|im_start|>assistant\n"

  # Format chosen answer
  chosen = example['chosen'] + "<|im_end|>\n"

  # Format rejected answer
  rejected = example['rejected'] + "<|im_end|>\n"

  return {"prompt":prompt,"chosen":chosen,"rejected":rejected}



In [ ]:
dpo_dataset=dpo_dataset.map(chat_format)

Map:   0%|          | 0/2365 [00:00<?, ? examples/s]

In [ ]:
dpo_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 2365
})

Load Model

In [ ]:
model_name="Porselvi/phi-2-finetuned"

In [ ]:

model=AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    load_in_4bit=True,
    trust_remote_code=True
    )
model.config.use_cache=False
#Reference model
ref_model=AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    load_in_4bit=True,
    trust_remote_code=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/898 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/559M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#tokenizer
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

tokenizer_config.json:   0%|          | 0.00/7.37k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
peft_config = LoraConfig(
r=32,
lora_alpha=16,
lora_dropout=0.05,
bias="none",
target_modules=['q_proj', 'k_proj', 'v_proj', 'dense', 'fc1', 'fc2'],
task_type="CAUSAL_LM",
)


In [ ]:
training_args=TrainingArguments(
    output_dir="phi-2",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    max_grad_norm=0.3,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    learning_rate=2e-4,
    save_steps=100,
    max_steps=400,
    gradient_checkpointing=True,
    gradient_accumulation_steps=4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.001,
    logging_steps=10
    )

In [ ]:
dpo_trainer = DPOTrainer(
    model,
    ref_model,
    force_use_ref_model=True,
    tokenizer=tokenizer,
    train_dataset=dpo_dataset,
    peft_config=peft_config,
    max_length=600,
    args=training_args,

)

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:307: UserWarning: `max_prompt_length` is not set in the DPOTrainer's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:332: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
Parameter 'function'=<bound method DPOTrainer.tokenize_row of <trl.trainer.dpo_trainer.DPOTrainer object at 0x7f47591725c0>> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous c

Map:   0%|          | 0/2365 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
dpo_trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
10,0.459000
20,0.120000
30,0.065500
40,0.053800
50,0.208400
60,0.006100
70,0.002400
80,0.022700
90,0.115300
100,0.052100


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

TrainOutput(global_step=400, training_loss=0.045323855373135304, metrics={'train_runtime': 1043.7237, 'train_samples_per_second': 1.533, 'train_steps_per_second': 0.383, 'total_flos': 0.0, 'train_loss': 0.045323855373135304, 'epoch': 0.68})

In [ ]:
dpo_trainer.model.save_pretrained("phi-2/checkpoint-400")
tokenizer.save_pretrained("phi-2/checkpoint-400")

('phi-2/checkpoint-400/tokenizer_config.json',
 'phi-2/checkpoint-400/special_tokens_map.json',
 'phi-2/checkpoint-400/vocab.json',
 'phi-2/checkpoint-400/merges.txt',
 'phi-2/checkpoint-400/added_tokens.json',
 'phi-2/checkpoint-400/tokenizer.json')

In [ ]:
#clear the memory
import gc
del model,dpo_trainer,ref_model
gc.collect()
torch.cuda.empty_cache()

Reload model and merge

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    return_dict=True,
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Merge base model with the adapter
model = PeftModel.from_pretrained(base_model, "phi-2/checkpoint-400")
model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model_path='Porselvi/phi-2-dpo-finetuned'
model.push_to_hub(model_path,token=True,safe_serialization=True)
tokenizer.push_to_hub(model_path,token=True,safe_serialization=True)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/559M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Porselvi/phi-2-dpo-finetuned/commit/5fc3780fc76bdd16581b60ad9c24b704cfde1709', commit_message='Upload tokenizer', commit_description='', oid='5fc3780fc76bdd16581b60ad9c24b704cfde1709', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Save model and tokenizer
#model.save_pretrained(model)
#tokenizer.save_pretrained(model)